In [1]:
import requests
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt
import json
import re
import os
import sys
from dotenv import load_dotenv
from datetime import datetime, timedelta
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util
sys.path.append(os.path.abspath(os.path.join('..', 'models')))
import nn
import torch

warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'nn'

In [ ]:
load_dotenv()
proj = hopsworks.login()
fs = proj.get_feature_store() 

In [ ]:
el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

In [ ]:
el_prices_fg.show(10)

In [ ]:
input_features = [
        "date", "temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max",
        "wind_direction_10m_dominant", "sunshine_duration",
        "Hydro Water Reservoir", "Nuclear", "Other", "Wind Onshore"
]
input_size = len(input_features)
model = nn.EnergyPricePredictorLSTM(input_size=input_size)


In [ ]:
all_data = 1 #placeholder
all_data = all_data.values

start_date = "2022-11-01"
end_date = "2024-12-22"

inputs, outputs = util.create_time_series(all_data)
#Split 70/15/15
num_samples = inputs.shape[0]
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)

train_inputs = inputs[:train_size]
train_outputs = outputs[:train_size]

val_inputs = inputs[train_size : train_size + val_size]
val_outputs = outputs[train_size : train_size + val_size]

test_inputs = inputs[train_size + val_size :]
test_outputs = outputs[train_size + val_size :]


#Tensors

train_data = torch.tensor(train_inputs, dtype=torch.float32)
train_labels = torch.tensor(train_outputs, dtype=torch.float32)
validation_data = torch.tensor(val_inputs, dtype=torch.float32)
validation_labels = torch.tensor(val_outputs, dtype=torch.float32)
test_data = torch.tensor(test_inputs, dtype=torch.float32)
test_labels = torch.tensor(test_outputs, dtype=torch.float32)




In [ ]:
model.train_model(train_inputs, train_outputs, val_inputs, val_outputs)

In [ ]:
predictions, mse = model.test_model(test_data, test_labels)
model.plot_predictions(predictions, test_labels)